In [9]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from io import StringIO
import time

In [6]:
year = 2022
url = f"https://www.pro-football-reference.com/years/{year}/coaches.htm" #can expand to other years later
response = requests.get(url)
if response.status_code == requests.codes.ok:
    soup = BeautifulSoup(response.text, features="html.parser")
else:
    response.raise_for_status()

table = soup.find('table', {'id':'coaches'})
coach_data = (
    pd.read_html(str(table))[0]
    .iloc[1:]
)
coach_data['year'] = year

HTTPError: 429 Client Error: Too Many Requests for url: https://www.pro-football-reference.com/years/2022/coaches.htm

In [4]:
def retrieve_year_coach_data(year):
    url = f"https://www.pro-football-reference.com/years/{year}/coaches.htm"
    response = requests.get(url)
    if response.status_code == requests.codes.ok:
        soup = BeautifulSoup(response.text, features="html.parser")
    else:
        response.raise_for_status()

    table = soup.find('table', {'id':'coaches'})
    coach_data = (
        pd.read_html(StringIO(str(table)))[0]
        .iloc[1:]
    )
    coach_data['year'] = year
    return coach_data

In [ ]:
def clean_coach_data_year(year):
    df = retrieve_year_coach_data(year)
    df.columns = df.columns.droplevel(0)
    df = (
        df[['Coach', 'Tm', '', 'G']]
        .rename(columns={'':'year'})
        .iloc[:, 0:4]
        .rename(columns={'Coach':'coach','Tm':'team', 'G':'num_games_coached'})
    )
    df = df.loc[df.groupby('team')['num_games_coached'].idxmax(), ['team', 'coach', 'year', 'num_games_coached']].reset_index(drop=True)
    return df

In [21]:
def get_all_coach_data(begin_range = 1999, end_range=2024):
    data_list = []
    years = range(begin_range, end_range)
    for year in years:
        data_list.append(clean_coach_data_year(year))
    return pd.concat(data_list)

In [ ]:
all_coach_data = get_all_coach_data()

HTTPError: 429 Client Error: Too Many Requests for url: https://www.pro-football-reference.com/years/2004/coaches.htm